In [ ]:
import pandas as pd
"""
header = 0 은 파일의 첫 줄에 열 이름이 있음을 의미
delimiter = \t 는 필드가 탭으로 구분됨을 의미
quoting = 3은 쌍따옴표 무시하는 태그
"""
#QOUTE_MINIMAL (0), QUOTE_ALL (1),
#QOUTE_NONNUMERIC (2) or QUOTE_NONE (3).

#Labeled data(for learning)
train = pd.read_csv('data/labeledTrainData.tsv', 
                    header = 0, delimiter='\t', quoting = 3)

#Test data(for testing)
test = pd.read_csv('data/testData.tsv',
                  header = 0, delimiter = '\t', quoting = 3)
train.shape
#25000개의 리뷰, 3개의 column

In [ ]:
train.head(4)
#train데이터의 일부 (위에서 4개를 미리 봄) tail은 반대
#senitment = 0: bad, 1: good

In [ ]:
train.columns.values

In [ ]:
train.describe()
#대략적인 데이터 설명

In [ ]:
#train['sentiment'].value_counts()
#Column 별 데이터 수 산출

train데이터에서 review 칼럼 첫번째 데이터를 700자까지만 출력
그 결과, html 태그를 정재할 필요가 있다.(text cleaning, preprocessing)
BeautifulSoup으로 HTML 태그 제거
알파벳 이외 문자를 공백으로 변경
NLTK를 이용해 불용어(stopword: I, my, me 같은 의미 없는 단어)를 제거
어간 추출(Stemming)과 음소표기법(Lemmatizing): SnowballStemmer로 어간 추출

한글 분석기: 트위터 형태소 분석기 (KoNLPy패키지에 포함됨)를 사용한다.

In [ ]:
#train['review'][0][:700]


In [ ]:
from bs4 import BeautifulSoup

example1 = BeautifulSoup(train['review'][0], "html.parser")
print(train['review'][0][:500])
example1.get_text()[:500]

#BeautifulSoup을 이용해 html 태그를 지운 결과

In [ ]:
# 정규표현식을 사용해서 특수문자를 제거
import re
#소문자와 대문자가 아닌 것은 공백으로 대체(^는 not을 의미)
letters_only = re.sub('[^a-zA-Z]',' ', example1.get_text())
letters_only[:500]

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle mess'

In [ ]:
#모두 소문자로 변환
lower_case = letters_only.lower()
#토큰화해서 words에 다 집어넣음
words = lower_case.split()
print(len(words))
words[:10]

437


['with',
 'all',
 'this',
 'stuff',
 'going',
 'down',
 'at',
 'the',
 'moment',
 'with']

# 불용어(stopword) 제거: NLTK 에서 가져옴

In [ ]:
import nltk
from nltk.corpus import stopwords
stopwords.words('english')[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

# 스태밍
#스태밍:어간 추출, 형태소 분석
#단어의 접사 등 필요없는 부분을 제외하고 어간만 분리 해내는 것
#복수형, 과거형, 명사형 등을 기본 단어(어간)으로 분리
#ex) fishing, fisher -> fish
#nltk안의 몇 가지 stemmer를 사용한다. stemmer마다 결과가 다르다.

In [ ]:

from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english')
words = [stemmer.stem(w) for w in words]

#여기서는 SnowballStemmer를 사용한다. 처리 결과는:
words[:10]

['with', 'all', 'this', 'stuff', 'go', 'down', 'at', 'the', 'moment', 'with']

# 음소표기법(Lemmatization):
'배'같은 동음 이의어를 문맥을 보고 식별할 수 있게 함.
동사로 쓰였는지, 명사로 쓰였는지 등을 통해 구분

In [ ]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

#예시:
print(wordnet_lemmatizer.lemmatize('fly'))
print(wordnet_lemmatizer.lemmatize('flies'))

words = [wordnet_lemmatizer.lemmatize(w) for w in words]
#처리 후 단어
words[:10]


fly
fly


['with', 'all', 'this', 'stuff', 'go', 'down', 'at', 'the', 'moment', 'with']

# 문자열 처리 함수
### 위의 기능들을 이용하여 문자열을 처리하는 함수를 별도로 만들어 본다.

In [ ]:
def review_to_words(raw_review):
    # 1. HTML 태그 제거
    review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
    # 2. 영문자 아닌 문자는 공백으로
    letters_only = re.sub('[^a-zA-Z]',' ', raw_review)
    # 3. 대문자를 소문자로 + 토큰화
    words = letters_only.lower().split()
    # 4. stopwords를 세트 형식으로 만들어서 찾게한다(리스트 보다 빨라서)
    stops = set(stopwords.words('english'))
    # 5. stopwords 찾아서 제거
    meaningful_words = [w for w in  words if not w in stops]
    # 6. 어간 추출(스태밍)
    stemming_words = [stemmer.stem(w) for w in meaningful_words]
    # 7. 공백으로 구분된 문자열로 결합하여 결과를 하나로 리턴
    return(' '.join(stemming_words))

In [ ]:
clean_review = review_to_words(train['review'][0])
#마이클 잭슨 관련 영화 리뷰만 한 번 해봄
clean_review

'stuff go moment mj start listen music watch odd documentari watch wiz watch moonwalk mayb want get certain insight guy thought realli cool eighti mayb make mind whether guilti innoc moonwalk part biographi part featur film rememb go see cinema origin releas subtl messag mj feel toward press also obvious messag drug bad kay br br visual impress cours michael jackson unless remot like mj anyway go hate find bore may call mj egotist consent make movi mj fan would say made fan true realli nice br br actual featur film bit final start minut exclud smooth crimin sequenc joe pesci convinc psychopath power drug lord want mj dead bad beyond mj overheard plan nah joe pesci charact rant want peopl know suppli drug etc dunno mayb hate mj music br br lot cool thing like mj turn car robot whole speed demon sequenc also director must patienc saint came film kiddi bad sequenc usual director hate work one kid let alon whole bunch perform complex danc scene br br bottom line movi peopl like mj one leve

In [ ]:
num_reviews = train['review'].size
num_reviews
#분석해야할 리뷰의 수 나열

25000

# for 문이나 기타 반복문으로는 오래 걸림 
25000개에 3분 정도 - > Multiprocessing 사용 시 45초 정도
라는데 무슨 이유에서인지 무한루프 돌아서 그냥 apply 사용
https://gist.github.com/yong27/7869662

In [ ]:

# 참고 : https://gist.github.com/yong27/7869662
# http://www.racketracer.com/2016/07/06/pandas-in-parallel/
from multiprocessing import Pool
import numpy as np

def _apply_df(args):
    df, func, kwargs = args
    return df.apply(func, **kwargs)

def apply_by_multiprocessing(df, func, **kwargs):
    # 키워드 항목 중 workers 파라메터를 꺼냄
    workers = kwargs.pop('workers')
    # 위에서 가져온 workers 수로 프로세스 풀을 정의
    pool = Pool(processes=workers)
    # 실행할 함수와 데이터프레임을 워커의 수 만큼 나눠 작업
    result = pool.map(_apply_df, [(d, func, kwargs)
            for d in np.array_split(df, workers)])
    pool.close()
    # 작업 결과를 합쳐서 반환
    return pd.concat(list(result))

In [ ]:
%time clean_test_reviews = apply_by_multiprocessing(\
    test['review'], review_to_words, workers=4)